In [42]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [46]:
# 데이터 로드
train_df = pd.read_csv("D:\\ML_kaggle\\data\\train_dataset.csv")
test_df = pd.read_csv("D:\\ML_kaggle\\data\\test_dataset.csv")
info_df = pd.read_csv("D:\\ML_kaggle\\data\\station_info.csv")

In [47]:
# 결측치 -9999 처리 (예시: NaN으로 변경 후 평균으로 채우기)
train_df = train_df.replace(-9999, np.nan)
test_df = test_df.replace(-9999, np.nan)

# 불필요한 컬럼 제거 (학습 데이터)
train_df = train_df.drop(columns=['station', 'station_name', 'date'], axis=1)
test_df = test_df.drop(columns=['station', 'station_name', 'date'], axis=1)

In [60]:
# 결측치 -9999 처리
train_df = train_df.replace(-9999, np.nan)
test_df = test_df.replace(-9999, np.nan)

In [61]:
# 시간별 변수 평균값으로 Feature 생성
# time_cols_train = [col for col in train_df.columns if '_' in col]
# train_df['avg_time_vars'] = train_df[time_cols_train].mean(axis=1)

# time_cols_test = [col for col in test_df.columns if '_n' in col]
# test_df['avg_time_vars'] = test_df[time_cols_test].mean(axis=1)

# NaN 값 채우기 (평균값으로 채우기)
# train_df = train_df.fillna(train_df.mean())
# test_df = test_df.fillna(test_df.mean())

# NaN 값 채우기 (최빈값으로 채우기)
for col in train_df.columns:
    mode_value = train_df[col].mode()
    if not mode_value.empty:
        train_df[col] = train_df[col].fillna(mode_value[0])
        continue
    train_df = train_df.drop(columns=[col], axis=1)

for col in test_df.columns:
    mode_value = test_df[col].mode()
    if not mode_value.empty:
        test_df[col] = test_df[col].fillna(mode_value[0])
        continue
    test_df = test_df.drop(columns=[col], axis=1)

In [62]:
# train_df의 컬럼별 NaN 값 개수 확인
# train_df의 총 NaN 값 개수 확인
total_nan_count = train_df.isnull().sum().sum()
print(f"train_df의 총 NaN 값 개수: {total_nan_count}")
total_nan_count = test_df.isnull().sum().sum()
print(f"test_df의 총 NaN 값 개수: {total_nan_count}")

train_df의 총 NaN 값 개수: 0
test_df의 총 NaN 값 개수: 0


In [65]:
# Feature 선택
features_train = [col for col in train_df.columns if col not in ['id', 'target','sunshine_duration_21']]
features_test = [col for col in test_df.columns if col not in ['id']]

train_only_features = set(features_train) - set(features_test)
print("Train에만 있는 Feature:", train_only_features)

# Feature 스케일링 (선택 사항)
scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[features_train])
y_train = train_df['target']
X_test = scaler.fit_transform(test_df[features_test])

print("X_train Feature 개수:", X_train.shape[1])
print("X_test Feature 개수:", X_test.shape[1])

Train에만 있는 Feature: set()
X_train Feature 개수: 328
X_test Feature 개수: 328


In [66]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [68]:
submission_df = pd.DataFrame({'id': test_df['id'], 'target': predictions})
submission_df


,id,target
0,0,-1.093202
1,1,2.146262
2,2,0.443306
3,3,2.553764
4,4,3.197479
...,...,...
4374,4374,-0.571236
4375,4375,-0.215027
4376,4376,1.373031
4377,4377,2.824722


In [69]:
submission_df.to_csv('submission.csv', index=False)